In [ ]:
import numpy as np
import pandas as pd
import re
import heapq
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Data preprocessing
with open("data\\txt\\master.txt", "r", encoding="utf-8") as infile:
    for line in infile:
        data_text = line

def text_cleaner(text):
    newString = text.lower()
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zñáéíóúü]", " ", newString)
    long_words=[]
    for i in newString.split():
        if len(i)>=3:
            long_words.append(i)
    return (" ".join(long_words)).strip()

data_new = text_cleaner(data_text)

# Creating Sequences
def create_seq(text, length = 30):
    sequences = list()
    for i in range(length, len(text)):
        seq = text[i-length:i+1]
        sequences.append(seq)
    return sequences

sequences = create_seq(data_new)

# Character Mapping
chars = sorted(list(set(data_new)))
mapping = dict((c, i) for i, c in enumerate(chars))

def encode_seq(seq):
    sequences = list()
    for line in seq:
        encoded_seq = [mapping[char] for char in line]
        sequences.append(encoded_seq)
    return sequences

sequences = encode_seq(sequences)

# Preparing the dataset
vocab = len(mapping)
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab)
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# Defining the model
model = Sequential()
model.add(Embedding(vocab, 50, input_length=30, trainable=True))
model.add(GRU(150, recurrent_dropout=0.1, dropout=0.1))
model.add(Dense(vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer=Adam(learning_rate=0.01))

# Training
callbacks = [EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint('model.h5', save_best_only=True,
             save_weights_only=False, monitor='val_loss')]

history = model.fit(X_tr, y_tr, epochs=50, batch_size=256,
                    verbose=1, callbacks=callbacks, validation_data=(X_val, y_val))

# Function for text generation using beam search
def generate_seq_beam_search(model, mapping, seq_length, seed_text, n_chars, beam_width=3):
    sequences = [{'seq': seed_text, 'score': 0.0}]
    for _ in range(n_chars):
        all_candidates = list()
        for i in range(len(sequences)):
            seq, score = sequences[i]['seq'], sequences[i]['score']
            encoded = [mapping[char] for char in seq]
            encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre').squeeze()
            pred = model.predict(np.array([encoded]), verbose=0)
            probas = np.exp(pred) / np.sum(np.exp(pred))
            top_k = heapq.nlargest(beam_width, zip(probas[0], list(range(len(probas[0])))))
            for j in range(len(top_k)):
                score_, idx = top_k[j]
                out_char = ''
                for char, index in mapping.items():
                    if index == idx:
                        out_char = char
                        break
                candidate = {'seq': seq + out_char, 'score': score - np.log(score_)}
                all_candidates.append(candidate)
        ordered = sorted(all_candidates, key=lambda tup:tup['score'])
        sequences = ordered[:beam_width]
    return sequences

# Generate text
print(generate_seq_beam_search(model, mapping, 30, "Domingo nueve de Junio de mil sete".lower(), 15))
